In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import InvalidSessionIdException
from selenium.common.exceptions import NoSuchWindowException

from urllib.parse import quote_plus    # 한글 텍스트를 퍼센트 인코딩으로 변환
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리
import pandas as pd
import timeit
import datetime

start_time = time.ctime()
start_time_sec = time.time()


driver = webdriver.Chrome("C:\informs\chromedriver.exe")
driver.implicitly_wait(3)

#페이지 띄우기
url = 'https://play.google.com/store/apps/details?id=kr.co.daelimcorp.smarthome' #'https://play.google.com/store/apps/details?id=com.mobiletalk.valley'
driver.get(url)
driver.implicitly_wait(5)

#모든페이지 클릭 클릭 
driver.find_element_by_css_selector('#fcxH9b > div.WpDbMd > c-wiz > div > div.ZfcPIb > div > div > main > div > div.W4P4ne > div.XnFhVd > div > span > span').click()
time.sleep(3)

SCROLL_PAUSE_TIME = 2

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom                                                      
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)                                                
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate new scroll height and compare with last scroll height            
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:                                                
        break
    
    last_height = new_height

time.sleep(3)

cnt = 1

try:
    while cnt <= 1:
        driver.find_element_by_css_selector('#fcxH9b > div.WpDbMd > c-wiz:nth-child(4) > div > div.ZfcPIb > div > div > main > div > div.W4P4ne > div:nth-child(2) > div.PFAhAf > div > span > span').click()
        time.sleep(3)
        
        SCROLL_PAUSE_TIME = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom                                                      
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)                                                
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height            
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:                                                
                break
            last_height = new_height
except NoSuchElementException:
    pass


review_html = driver.page_source
review_soup = BeautifulSoup(review_html, 'html.parser')
#업소명

reviews = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']")
len(reviews)

data = []


for i in range(len(reviews)):
    
    paragraph = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[i]
    
    if i%50 == 0: 
        print('크롤링 되고 있는 수 :',i)
    
    #paragraph0 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[0]
    #paragraph1 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[1]
    #paragraph2 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[2]
    #paragraph3 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[3]
    #paragraph4 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[4]
    #paragraph5 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[5]
    #paragraph6 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[6]
    #paragraph7 = review_soup.find_all("div", {"class":"d15Mdf bAhLNe"})[7]
    
    title = review_soup.find("meta", {"property":"og:title"}).get('content')
    score = paragraph.find_all("div", {"class":"vQHuPe bUWb7c"})
    dates = paragraph.find_all("span", {"class":"p2TkOb"})
    names = paragraph.find_all("span", {"class":"X43Kjb"})
    reviews = paragraph.find_all("span", {"jsname":"bN97Pc"})
    
    
    num = len(score)
    
    tmp_date = []
    
    for date in dates:
        date_list = date.get_text()
        tmp_date.append(date_list)
    
        
    tmp_name = []
    
    for name in names:
        a = name.get_text()
        tmp_name.append(a)
    
    tmp_review = []
    
    for review in reviews:
        review_list = review.get_text()
        tmp_review.append(review_list)
    
    if len(tmp_name) ==2:
        data.append({'title': title,
                     'score': num,
                    'date': tmp_date[0],
                    'writer': tmp_name[0],
                    'review': tuple(tmp_review),
                    'replied_date': tmp_date[1], 
                    'replier': tmp_name[1]})
    else:
        data.append({'title': title,
                     'score': num,
                    'date': tmp_date[0],
                    'writer': tmp_name[0],
                    'review': tuple(tmp_review),
                    'replied_date': '대댓글없음',
                    'replier': '대댓글없음'})
        

end_time = time.ctime()    
end_time_sec = time.time()


sec = end_time_sec - start_time_sec

time = str(datetime.timedelta(seconds=sec)).split(".")
times = time[0]
print(times)

print('시작시간은:', start_time)
print('끝난시간은:', end_time)

res_df = pd.DataFrame(data)
res_df

크롤링 되고 있는 수 : 0
크롤링 되고 있는 수 : 50
크롤링 되고 있는 수 : 100
크롤링 되고 있는 수 : 150


In [244]:
import datetime
import numpy as np

res_df = pd.DataFrame(data)
#res_df
df = res_df.drop_duplicates()

df['title'] = df['title'].replace(' - Google Play 앱', '', regex=True)

df['date'] = df['date'].replace('년 ', '-', regex=True)
df['date'] = df['date'].replace('월 ', '-', regex=True)
df['date'] = df['date'].replace('일', '', regex=True)
df['replied_date'] = df['replied_date'].replace('년 ', '-', regex=True)
df['replied_date'] = df['replied_date'].replace('월 ', '-', regex=True)
df['replied_date'] = df['replied_date'].replace('일', '', regex=True)
df['replied_date'] = df['replied_date'].replace('대댓글없음', '1900-1-1', regex=True)

df['date_type_chnged'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['replied_date_type_chnged'] = pd.to_datetime(df['replied_date'], format='%Y-%m-%d')
df['time_diff'] = df['replied_date_type_chnged'] - df['date_type_chnged']
df['time_diff_type_chnged'] = (df['time_diff'] / np.timedelta64(1,'D')).astype(int)
#df.dtypes
df

<ipython-input-244-48bbdb95f5dd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].replace(' - Google Play 앱', '', regex=True)
<ipython-input-244-48bbdb95f5dd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].replace('년 ', '-', regex=True)
<ipython-input-244-48bbdb95f5dd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,title,score,date,writer,review,replied_date,replier,date_type_chnged,replied_date_type_chnged,time_diff,time_diff_type_chnged
0,아이파크 스마트홈2.0,1,2020-10-5,impact II,"(다른 분들 얘기하시는 단점 모두 다 여전히 고쳐지지 않았고(자동 로그아웃,각 방별...",1900-1-1,대댓글없음,2020-10-05,1900-01-01,-44107 days,-44107
1,아이파크 스마트홈2.0,1,2020-10-7,박성현,(로그인은 서버에서 세션ID를 클라이언트로 발급하고 앱은 그 세션ID를 쿠키로 저장...,1900-1-1,대댓글없음,2020-10-07,1900-01-01,-44109 days,-44109
2,아이파크 스마트홈2.0,1,2020-9-30,한승주,(스마트홈이라는데 시간되면 꺼지는/켜지는 기능도 없네요. 매번 로그인하는 것도 귀찮...,1900-1-1,대댓글없음,2020-09-30,1900-01-01,-44102 days,-44102
3,아이파크 스마트홈2.0,1,2020-9-17,홍정우,(다른분들도 써주셨지만 거실빼곤 어디 콘센트이고 어디 조명인지가 적혀있지 않아서 쓰...,1900-1-1,대댓글없음,2020-09-17,1900-01-01,-44089 days,-44089
8,아이파크 스마트홈2.0,3,2020-10-7,Seokyoung Lee,"(폰 변경 전에는 자동차 입차 시 푸시 알림이 왔으나, 폰 변경 후에는 푸시 알림이...",1900-1-1,대댓글없음,2020-10-07,1900-01-01,-44109 days,-44109
9,아이파크 스마트홈2.0,1,2020-10-26,타우린존버,"(갑자기 우리집제어(전기 콘센트 환풍기 난방)제어 기능이 사라지고 외출설정만 됩니다,)",1900-1-1,대댓글없음,2020-10-26,1900-01-01,-44128 days,-44128
10,아이파크 스마트홈2.0,3,2020-10-26,SeongBeom Ahn,(조명 콘센트 온도설정할때 방이름이 없어서 방이름을 설정하거나 방넘버가 적혀있었으면...,1900-1-1,대댓글없음,2020-10-26,1900-01-01,-44128 days,-44128
11,아이파크 스마트홈2.0,2,2020-10-24,SHIN MH,"(스위치, 난방, 조명 각실 명칭이 있어야 분간이 될것 같아요,)",1900-1-1,대댓글없음,2020-10-24,1900-01-01,-44126 days,-44126
12,아이파크 스마트홈2.0,1,2020-6-18,Sejoon Min,(기능에 방정보가 없어서 어떤 버튼이 어느방인지 알수가없어요 기능도 월패드 기능전부...,1900-1-1,대댓글없음,2020-06-18,1900-01-01,-43998 days,-43998
13,아이파크 스마트홈2.0,1,2020-8-14,hs son,(업뎃이안되나봐요;; 제어하려해도 위치가어디인지도 안나오고 찾아서 적어놓고싶어도 그...,1900-1-1,대댓글없음,2020-08-14,1900-01-01,-44055 days,-44055


In [245]:
is_re_reply = df['replier'] != '대댓글없음'
df_v1= df[is_re_reply]
#print(df_v1)

print('전체 댓글수는 ',len(df),'이고 그 중 대댓글이 있는 수는 ', len(df_v1), '이며 대댓글이 남겨지는 평균일수는', round(sum(df_v1['time_diff_type_chnged'])/len(df_v1),1),'일 이다.')

is_score_5 = df['score'] == 5
is_score_4 = df['score'] == 4
is_score_3 = df['score'] == 3
is_score_2 = df['score'] == 2
is_score_1 = df['score'] == 1

df_score_5= df[is_score_5]
df_score_4= df[is_score_4]
df_score_3= df[is_score_3]
df_score_2= df[is_score_2]
df_score_1= df[is_score_1]

print('전체 댓글수는 ',len(df),'중 별점 5개는 ', len(df_score_5), '개 이고 별점 4개는 ',len(df_score_4),'개 이고 별점 3개는 ',len(df_score_3),'개 이고 별점 2개는 ',len(df_score_2),'개 이고 별점 1개는 ',len(df_score_1),'개 이다.','(각 5개', round(len(df_score_5)/len(df),2),'%, 4개',round(len(df_score_4)/len(df)*100,2),'%, 3개',round(len(df_score_3)/len(df)*100,2),'%, 2개',round(len(df_score_2)/len(df)*100,2),'%, 1개',round(len(df_score_1)/len(df)*100,1),'%)')


ZeroDivisionError: division by zero

In [242]:
#!pip install wordcloud
